In [50]:

import json
import logging
import os
from typing import List, Tuple, Dict, Any
import pddl
from pddl.core import Domain, Problem

DB_FILE_PATH = "data/domains.db"
CONFIG_FILE_PATH = "data/config.json"
TEMPLATE_FILE_NAME = "template.pddl.txt"
GROUND_TRUTH_FILE_NAME = "ground.pddl"
NL_FILE_NAME = "nl.json"

def get_new_domains() -> list[str]:
    """returns a list of paths of folders in data/domains marked as new in data/config.json"""
    with open(CONFIG_FILE_PATH, "r") as f:
        config = json.load(f)
    new_domains = []
    for domain in config["new_domains"]:
        domain_path = os.path.join("data/domains", domain)
        if os.path.isdir(domain_path):
            new_domains.append(domain_path)
        else:
            logging.warning(f"Domain path {domain_path} is listed as a new domain in data/config.json but does not exist.")
    return new_domains

def parse_plan(plan_path: str) -> List[Tuple[str, List[str]]]:
    """Parses a plan file into a list of actions and their parameters."""
    with open(plan_path, "r") as f:
        lines = f.readlines()
    plan = []
    for line in lines:
        line = line.strip()
        if line.startswith("(") and line.endswith(")"):
            action = line[1:-1].split()
            action_name = action[0]
            parameters = action[1:]
            plan.append((action_name, parameters))
    return plan

# Relative paths to domain folders    
domain_paths = get_new_domains()

# Dictionary from domain folders to an AST Domain object representing the 
# Ground truth domain in the folder
domains : Dict[str, Domain] = []

# Dictionary from domain paths to a list of paths to problem files in the folder
problem_paths : Dict[str, List[str]] = {}

# Dictionary from problem paths to an AST Problem object
problems : Dict[str, Problem] = {}

# Dictionary from problem paths to for now a single plan paths.
plan_paths : Dict[str, str] = {}

# "AST" of the plan, maps a single plan path to its "AST", a list of actions and their arguments.
plans : Dict[str, List[Tuple[str, List[str]]]] = {}

# NL descriptions of the predicates and actions in the domain
nl_json : Dict[str, Any] = {}

#Array of every file with the name p*.pddl in the domain folder, use regex to get the name of the problem
for domain_path in domain_paths: 
    domain = pddl.parse_domain(os.path.join(domain_path, GROUND_TRUTH_FILE_NAME))
    problem_paths[domain_path] = [os.path.join(domain_path, f) for f in os.listdir(domain_path) if f.startswith("p") and f.endswith(".pddl")]
    for problem_file in problem_paths[domain_path]:
        problem = pddl.parse_problem(problem_file)
        problems[problem_file] = problem
        plan_paths[problem_file] = problem_file.replace(".pddl", ".plan.txt")
        plans[problem_file] = parse_plan(plan_paths[problem_file])
    with open(os.path.join(domain_path, NL_FILE_NAME), "r") as f:
        nl_json[domain_path] = json.load(f)

print(domain_paths)
print(problem_paths)
print(plan_paths)
#print(nl_json)


['data/domains/bookseller', 'data/domains/flow', 'data/domains/pathfinder']
{'data/domains/bookseller': ['data/domains/bookseller/p1.pddl'], 'data/domains/flow': ['data/domains/flow/p1.pddl', 'data/domains/flow/p2.pddl'], 'data/domains/pathfinder': ['data/domains/pathfinder/p1.pddl']}
{'data/domains/bookseller/p1.pddl': 'data/domains/bookseller/p1.plan.txt', 'data/domains/flow/p1.pddl': 'data/domains/flow/p1.plan.txt', 'data/domains/flow/p2.pddl': 'data/domains/flow/p2.plan.txt', 'data/domains/pathfinder/p1.pddl': 'data/domains/pathfinder/p1.plan.txt'}


In [51]:
def get_init_preds(problem: Problem) -> List[Tuple[str, List[str]]]:
    """Returns a set of predicates in the initial state of the problem."""
    return [(pred.name, [arg.name for arg in pred.terms]) for pred in problem.init]

def get_goal_preds(problem: Problem) -> List[Tuple[str, List[str]]]:
    """Returns a set of predicates in the goal state of the problem."""
    return [(pred.name, [arg.name for arg in pred.terms]) for pred in problem.init]

def get_all_preds(problem: Problem) -> List[Tuple[str, List[str]]]:
    """Returns a list of predicates in the initial state and goal of the problem."""
    init_preds = get_init_preds(problem)
    goal_preds = get_goal_preds(problem)
    return init_preds + goal_preds

def get_all_pred_names(problem: Problem) -> set[str]:
    """Returns a list of predicates in the initial state and goal of the problem."""
    return {pred[0] for pred in get_all_preds(problem)}

def get_all_pred_names_domain(domain_path: str) -> set[str]:
    """For a given domain, get the list of all predicates in the initial state and goal of all problems."""
    all_pred_names = set()
    for problem_path in problem_paths[domain_path]:
        all_pred_names= all_pred_names.union(get_all_pred_names(problems[problem_path]))
    return all_pred_names


In [ ]:

import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it in your .env file.")

# Provider model dictionary
MODELS = {
    "gpt-4o-mini", "gpt-4", "o4-mini"
}

models = {(model_name, init_chat_model(model_name)) for model_name in MODELS}










for domain_path in domain_paths:
    init_preds = get_all_pred_names_domain(domain_path)
    



